In [ ]:
import pdfplumber
import pandas as pd
import os
import spacy 
import numpy as np
from IPython.display import clear_output
import re

In [55]:
def get_names(dirname):
    """
    Get employee names, store ids and filenames
    """
    employee_names = []
    store_ids = []
    filepaths = []
    uids = []
    for root, dirs, files in os.walk(dirname):
        for item in files:
            if ".pdf" not in item:
                skip=True
                continue
            else:
                filename = os.path.join(root, item)
                filepaths.append(filename)
                item = re.sub('[\s]+', ' ', 
                              re.sub('^\s\-\s', ' - ', 
                                     re.sub('[\(\)]+', ' - ', item)))
                item = re.sub(' - not formatted', ' ', item)
                item = re.sub('(\s\-)[A-Z0-9]', ' - ', item)
                item = re.sub('.pdf', ' ', item)
                item = re.sub('[\s]+', ' ', item)
                #id = re.findall('[A-Z0-9]{3}', item)
                splits = item.split(" - ")
                employee_name = splits[0]
                print(employee_name)
                #print(item)
                if len(splits) != 1:
                    #print(item)
                    store_id = splits[1]
                    if len(splits) == 3:
                        uid = splits[2]
                        print(employee_name, "don't have uid")
                    else: uid = ''
                    #print(store_id)
                else: 
                    store_id = ''
                    print(employee_name, "don't have store id")
                print(uid)
                employee_names.append(employee_name)
                store_ids.append(store_id) 
                uids.append(uid)
    #print("get names: finished")
    return employee_names, store_ids, filepaths, uids

In [56]:
def parse_text(pdf):
    """
    Parse text in pdf resume
    """
    targets = []
    wordsdic = {}
    others = []
    index = dict()
    
    words_per_page = dict()
    for k in range(len(pdf.pages)):
        words = pdf.pages[k].extract_words(x_tolerance=5) # have a look at this hyperparam
        words_per_page[k]=len(words)
        try:
            targets.append(words[0]['text'])
        except IndexError:
            continue
            print(words)
        for i in range(1, len(words)):
            
            if words[i]["top"] - words[i-1]["top"] > 2:
                #print(words[i])
                targets.append("\n")
                targets.append(words[i]['text'])
            else:
                targets.append(words[i]['text'])
        #t = " ".join(targets).split("\n")
    return " ".join(targets).split("\n")

In [57]:
def read_pdf(path):
    pdf = pdfplumber.open(path)
    return pdf

In [58]:
def convert_to_table(dirname):
    dirpath = dirname
    employee_names, store_ids, filepaths, uids = get_names(dirpath)
    all_dict = {"employee_name":[], "store":[], "resume":[], "uid":[], "filename":[]}
    useless = 0
    useless_lst = []
    for i in range(len(filepaths)):
        try:
            clear_output(wait=True)
            employee_name = employee_names[i]
            store_id = store_ids[i]
            uid = uids[i]
            path = filepaths[i]
            pdf = read_pdf(path)
            text0 = parse_text(pdf)
            if "Scanned by CamScanner" in text0 or text0 == [""] or "Scanned by CamScanner" in text0[0]:
                useless += 1
                useless_lst.append(employee_name)
            all_dict["employee_name"].append(employee_name)
            all_dict["store"].append(store_id)
            all_dict["resume"].append(text0)
            all_dict["uid"].append(uid)
            all_dict["filename"].append(path)
            print("Current progress:", i, "/", len(filepaths)-1, np.round((i+1)/len(filepaths)*100, 2), "%")
            print("useless (scanned):", str(useless), "/", len(filepaths)-1, np.round(useless/len(filepaths)*100, 2), "%")
            print("finished", employee_name)
            print("scanned:", useless_lst)
        except TypeError:
            continue
    return pd.DataFrame(all_dict)

In [59]:
# run
dirpath = "../Career Builder (Resumes)/"
df = convert_to_table(dirpath)

Current progress: 88 / 88 100.0 %
useless (scanned): 12 / 88 13.48 %
finished Akerley, Daniel
scanned: ['Ayse, Seyli', 'Girard, Olivier', 'Munguia Garcia, Rafael Antonio', 'Levesque, Vianney', 'Sukhiani, Salman', 'Aguiar, Daniel', 'Abualkhair, Mohd', 'Gandhi, Nidhi', 'Hayward, Sheldon', 'Guilbault, Derek', 'Bernard, David', 'Gonzague, Aaron']


In [60]:
df # -> should then filter scanner 

,employee_name,store,resume,uid,filename
0,"Waldie, Megan",WW 120,"[Megan Waldie , 2110 EIGHTH AVENUE , NEW WES...",,"../Career Builder (Resumes)/Waldie, Megan - WW..."
1,"Xu, Yinyin",TB 100,"[Yinyin Xu , 201-3614 Richmond Road Cell: 778...",,"../Career Builder (Resumes)/Xu, Yinyin - TB 10..."
2,"Mahesaniya, Shanil Allauddin",WE 548,"[SHANIL MAHESANIYA , 4266 Grange Street, Burn...",,"../Career Builder (Resumes)/Mahesaniya, Shanil..."
3,"Hamel, Alexandre",WW 682,"[ALEXANDRE HAMEL , 819-635-4344 ahamel.tandem...",,"../Career Builder (Resumes)/Hamel, Alexandre -..."
4,"Ayse, Seyli",TB 283,[Scanned by CamScanner Scanned by CamScanner S...,,"../Career Builder (Resumes)/Ayse, Seyli - TB 2..."
...,...,...,...,...,...
83,"Ajmal, Syed Sheroze",TB 283,"[Sheroze Ajmal, Syed , 103 Cumberland Ave Sou...",,"../Career Builder (Resumes)/Ajmal, Syed Sheroz..."
84,"Belanger, Steve",WW 688,"[Steve Bélanger , 40 rue Saint-Joseph , Beau...",,"../Career Builder (Resumes)/Belanger, Steve - ..."
85,"Gonzague, Aaron",TB 426,[Scanned by CamScanner Scanned by CamScanner],,"../Career Builder (Resumes)/Gonzague, Aaron - ..."
86,"Malik, Divyanshu",TB 103,"[Divyanshu Malik divyanshu.malik06@gmail.com ,...",,"../Career Builder (Resumes)/Malik, Divyanshu -..."


In [ ]:
def filter_letters(lst):
    for item in lst:
        re.sub([A-Za-z])

In [178]:
d = ' Page 1 of 2 EDUCATION '
re.sub("Page [0-9]{1} of [0-9]{1}", "", d)

'  EDUCATION '

In [333]:
df.resume[83]

['Sheroze Ajmal, Syed ',
 ' 103 Cumberland Ave South, Saskatoon, SK ',
 ' 306-450-1472; sherozeajmal122@gmail.com ',
 ' Skills Profile ',
 ' • Strong administrative and organizational skills with a high level of enthusiasm and energy ',
 ' • 4 years+ of working experience with reputed organizations as a customer / client service ',
 ' representative ',
 ' • Ability to work in a fast paced environment, can work individually, as well as a team member ',
 ' of diversified groups of people ',
 ' • Result oriented, consistently get positive result from post assessment of clients ',
 ' • Having command over communication skills to actively listen, acknowledge concerns, and ',
 ' ability to provide explanations to resolve issues ',
 ' • Strong knowledge for customer services, and able to deal with difficult people in a ',
 ' professional manner ',
 ' • Strong knowledge of Microsoft Office Word, Excel and PowerPoint ',
 ' Work Experience ',
 ' The Source ',
 ' Customer Service Representative O

# Start feature extraction - work part 

In [334]:
# work-education segmentation 
x = df.resume[83]
for i in range(len(x)):
    s = re.sub("Page [0-9]{1} of [0-9]{1}", "", x[i])
    s = re.sub("[\s]+", " ", s)
    if len(s) < 20:
        if "work" in s.lower()  or "EXPERIENCE" in s or "Experience" in s or "EMPLOYMENT" in s:
            print(i, s)
        if "education" in s.lower():
            print(i, s)
        #else:
        #    print(i, s)

15  Work Experience 
51  Education: 


In [335]:
# use index
y = x[15:51]

In [336]:
import spacy

In [337]:
nlp = spacy.load("en_core_web_sm")

In [338]:
#doc = nlp(y)
#for ent in doc.ents:
#    print(ent.label_)

In [343]:
# locate and print out each work experience line 
for i in range(len(y)):
    #print(y[i])
    if len(y[i]) < 70:
        doc = nlp(y[i])
        for ent in doc.ents:
            if ent.label_ == "DATE":
                print(y[i])
                if len(y[i-1]) < 50:
                    print(y[i-1])
                break
    print("\n")





 Customer Service Representative October 2018 - May 2019 
 The Source 




































 CLS (Distribution Centre), Regina, SK April 2016 - Jan 2018 
 obtain in-depth knowledge for business 


















 Kognitive Marketing (P/T), Regina, SK April 2016 - Aug 2016 
 concerns with employees’ health 








 • Reported to manager regarding sales results on monthly basis 








In [226]:
for string in y:
    doc = nlp(string)
    for ent in doc.ents:
        #if ent.label_ == "ORG":
        #   print(ent.text, ent.label_)
        #if ent.label_ == "DATE":
        #    print(ent.text, ent.label_)

The Mobile Shop - Peterborough ORG
April 2018 DATE
February 2014 DATE
Serve ORG
Richmond Hill GPE
July 2012 to August 2012 DATE
Greet PERSON
August 2014 DATE
5 years DATE
Sara Jackson PERSON
371 CARDINAL
Ontario GPE
Canada GPE
July 7th DATE
2018 DATE
summer DATE
21 year old DATE
The Nutty Chocolatier ORG
Play Stables PERSON
Sara Jackson PERSON
